<a href="https://colab.research.google.com/github/Emmanuel-Ayeni/foulMvSoccerNet-/blob/main/ai_var_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install soccernet
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.1/922.1 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [3]:
!pwd

/content


In [4]:
!ls -lt

total 8
drwx------ 6 root root 4096 Feb 28 19:28 drive
drwxr-xr-x 1 root root 4096 Feb 27 14:22 sample_data


In [8]:
!ls -lt

total 8
drwx------ 6 root root 4096 Feb 28 19:28 drive
drwxr-xr-x 1 root root 4096 Feb 27 14:22 sample_data


In [9]:
# Libaries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import os
import zipfile
import json

import cv2  # OpenCV for video processing
import SoccerNet
import torch
from ultralytics import YOLO


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Example model
from sklearn.metrics import classification_report, accuracy_score

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [10]:
from SoccerNet.Downloader import SoccerNetDownloader as SNdl
mySNdl = SNdl(LocalDirectory="path/to/SoccerNet")
mySNdl.downloadDataTask(task="mvfouls", split=["train","valid","test","challenge"], password="s0cc3rn3t")

In [14]:
!ls -lt path/to/SoccerNet/mvfouls/


total 3246236
-rw-r--r-- 1 root root  245990818 Feb 28 21:48 challenge.zip
-rw-r--r-- 1 root root  268201537 Feb 28 21:35 test.zip
-rw-r--r-- 1 root root  350517090 Feb 28 21:34 valid.zip
-rw-r--r-- 1 root root 2459423137 Feb 28 21:34 train.zip


In [15]:
def extract_zip_files_to_subdirs(zip_dir, base_extract_dir):
    """
    Extracts each zip file in zip_dir to a subdirectory
    named after the zip file (without the .zip extension)
    within base_extract_dir.  No files are extracted directly
    to base_extract_dir.
    """

    try:
        for filename in os.listdir(zip_dir):
            if filename.endswith('.zip'):
                zip_filepath = os.path.join(zip_dir, filename)
                subdir_name = os.path.splitext(filename)[0]  # Name from filename
                extract_to = os.path.join(base_extract_dir, subdir_name)

                # Create the base directory if it doesn't exist.
                if not os.path.exists(base_extract_dir):
                    os.makedirs(base_extract_dir)

                if not os.path.exists(extract_to):
                    os.makedirs(extract_to)  # Create subdir if it doesn't exist.

                print(f"Extracting {filename} to {extract_to}...")

                with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)

                print(f"Extracted {filename} to {extract_to}")

    except Exception as e:
        print(f"Error during extraction: {e}")




In [16]:
zip_path = "./path/to/SoccerNet/mvfouls/"  # Replace with actual path
base_extract_dir = "./soccer_extract_data"  # Base directory for subfolders
extract_zip_files_to_subdirs(zip_path, base_extract_dir)

Extracting valid.zip to ./soccer_extract_data/valid...
Extracted valid.zip to ./soccer_extract_data/valid
Extracting test.zip to ./soccer_extract_data/test...
Extracted test.zip to ./soccer_extract_data/test
Extracting challenge.zip to ./soccer_extract_data/challenge...
Extracted challenge.zip to ./soccer_extract_data/challenge
Extracting train.zip to ./soccer_extract_data/train...
Extracted train.zip to ./soccer_extract_data/train


In [18]:
!ls -lt ./soccer_extract_data

total 120
drwxr-xr-x 2918 root root 77824 Feb 28 22:27 train
drwxr-xr-x  275 root root 12288 Feb 28 22:26 challenge
drwxr-xr-x  303 root root 12288 Feb 28 22:26 test
drwxr-xr-x  413 root root 16384 Feb 28 22:26 valid


In [19]:
!ls -lt ./soccer_extract_data/test/ # Unix command to verify the action directories and respective annotation json file

total 1552
-rw-r--r-- 1 root root 353861 Feb 28 22:26 annotations.json
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_99
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_98
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_97
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_96
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_95
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_94
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_93
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_92
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_91
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_90
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_9
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_89
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_88
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_87
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_86
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_85
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_84
drwxr-xr-x 2 root root   4096

In [20]:
!ls -lt ./soccer_extract_data/train/ # Unix command to verify the action directories and respective annotation json file

total 14948
-rw-r--r-- 1 root root 3360392 Feb 28 22:27 annotations.json
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_999
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_998
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_997
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_996
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_995
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_994
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_993
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_992
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_991
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_990
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_99
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_989
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_988
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_987
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_986
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_985
drwxr-xr-x 2 root root    4096 Feb 28 22:27 acti

In [21]:
!ls -lt ./soccer_extract_data/valid/ # Unix command to verify the action directories and respective annotation json file

total 2120
-rw-r--r-- 1 root root 483560 Feb 28 22:26 annotations.json
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_99
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_98
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_97
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_96
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_95
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_94
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_93
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_92
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_91
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_90
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_9
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_89
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_88
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_87
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_86
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_85
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_84
drwxr-xr-x 2 root root   4096

In [22]:
!ls -lt ./soccer_extract_data/challenge/ # Unix command to verify the action directories and except the annotation json file

total 1092
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_99
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_98
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_97
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_96
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_95
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_94
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_93
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_92
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_91
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_90
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_9
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_89
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_88
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_87
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_86
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_85
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_84
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_83
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_82
drwxr-xr-x 2 root roo

In [24]:
# Path to Video and JSON Annotation
video_path = "./soccer_extract_data/test/action_1.mp4"  # Replace with actual path
json_path = "./soccer_extract_data/test/annotations.json"  # Replace with actual path

In [25]:
# Load JSON Annotations
with open(json_path, 'r') as f:
    annotations = json.load(f)

In [28]:
# Video Processing (Frame Extraction and Saving)
def process_video(video_path, output_dir, frame_rate=5):  # Adjust frame rate as needed
    """Extracts frames from a video and saves them as images."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_subdir = os.path.join(output_dir, video_name)
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    success, frame = cap.read()

    while success:
        if frame_count % frame_rate == 0: # Extract every 'frame_rate'-th frame
            frame_filename = f"frame_{frame_count:05d}.jpg"  # Format frame number
            frame_path = os.path.join(output_subdir, frame_filename)
            cv2.imwrite(frame_path, frame)  # Save the frame

        frame_count += 1
        success, frame = cap.read() # Read the next frame
    cap.release()
    print(f"Extracted frames from {video_path} to {output_subdir}")
    return output_subdir



In [29]:
output_frames_dir = "./frames" # Directory to save extracted frames
frames_directory = process_video(video_path, output_frames_dir)

Extracted frames from ./soccer_extract_data/test/action_1.mp4 to ./frames/action_1


In [32]:
!ls -lt ./frames/

total 4
drwxr-xr-x 2 root root 4096 Feb 28 22:52 action_1


In [41]:
!ls -lt ./frames/action_1/

total 0


Data Proprocessing

In [40]:
import tensorflow
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load a pre-trained ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(video_frames):
    """
    Extract features from a sequence of video frames using ResNet50.
    """
    features = []
    for frame in video_frames:
        frame = cv2.resize(frame, (224, 224))  # Resize to ResNet input size
        frame = preprocess_input(frame)  # Preprocess for ResNet
        frame_features = base_model.predict(np.expand_dims(frame, axis=0))
        features.append(frame_features)
    return np.array(features)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Extract Frames from Videos
**bold text**

In [43]:
train_video_dir = "soccer_extract_data/train"

In [ ]:
def extract_frames(video_path, output_dir, frame_rate=5):
    """
    Extract frames from a video at a specified frame rate.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame at the specified frame rate
        if frame_count % frame_rate == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    return saved_count


In [44]:

# Example: Extract frames for all videos in the train directory
train_video_dir = "soccer_extract_data/train"
train_frame_dir = "soccer_extract_data/train/frames"

for video_file in os.listdir(train_video_dir ):
    video_path = os.path.join(train_video_dir, video_file)
    output_dir = os.path.join(train_frame_dir, os.path.splitext(video_file)[0])
    extract_frames(video_path, output_dir)

In [47]:
!ls -lt soccer_extract_data/train/frames

total 11668
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1208
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1292
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_332
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1064
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_152
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2136
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2581
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_379
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_565
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1042
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1241
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1896
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1898
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2167
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2837
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1399
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1581
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1707
drwxr-xr-x 2 root root 4096 Mar  1 01:

In [45]:
def load_annotations(annotation_path):
    """
    Load annotations from the JSON file.
    """
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations

In [46]:
# Example: Load train annotations
train_annotations = load_annotations("soccer_extract_data/train/annotations.json")

In [48]:
train_annotations

{'Set': 'train',
 'Number of actions': 2916,
 'Actions': {'0': {'UrlLocal': 'england_epl\\2014-2015\\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley',
   'Offence': 'Offence',
   'Contact': 'With contact',
   'Bodypart': 'Upper body',
   'Upper body part': 'Use of shoulder',
   'Action class': 'Challenge',
   'Severity': '1.0',
   'Multiple fouls': '',
   'Try to play': '',
   'Touch ball': '',
   'Handball': 'No handball',
   'Handball offence': '',
   'Clips': [{'Url': 'Dataset/Train/action_0/clip_0',
     'Camera type': 'Main camera center',
     'Timestamp': 1730826,
     'Replay speed': 1.0},
    {'Url': 'Dataset/Train/action_0/clip_1',
     'Camera type': 'Close-up player or field referee',
     'Timestamp': 1744173,
     'Replay speed': 1.8}]},
  '1': {'UrlLocal': 'england_epl\\2014-2015\\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley',
   'Offence': 'Offence',
   'Contact': 'With contact',
   'Bodypart': 'Under body',
   'Upper body part': '',
   'Action class': 'Tackling',
   'Severity': '

**[Feature Extraction](https://)**
Using a Pre-trained Model (ResNet50) to extract features from the frames

In [49]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(frame_paths):
    """
    Extract features from a list of frame paths.
    """
    features = []
    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        frame = cv2.resize(frame, (224, 224))  # Resize to ResNet input size
        frame = preprocess_input(frame)  # Preprocess for ResNet
        frame_features = base_model.predict(np.expand_dims(frame, axis=0))
        features.append(frame_features)
    return np.array(features)



In [51]:
# Example: Extract features for a video
video_frame_dir = "soccer_extract_data/train/frames/action_1"
frame_paths = [os.path.join(video_frame_dir, f) for f in os.listdir(video_frame_dir)]
video_features = extract_features(frame_paths)

In [52]:
import json
import os

def load_annotations(annotation_path):
    """
    Load annotations from the JSON file.
    """
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations



In [55]:
# Example: Load train annotations
train_annotation_path = "soccer_extract_data/train/annotations.json"
train_annotations = load_annotations(train_annotation_path)

# Get the keys from the train_annotations dictionary
keys = list(train_annotations.keys())

# Print an example annotation using the first key
if keys:  # Check if keys exist
    first_key = keys[0]
    print(f"Example annotation for {first_key}:", train_annotations[first_key])
else:
    print("No annotations found in the file.")

Example annotation for Set: train


Extract Frames from Videos
Extra frames from each video at a specified frame rate

In [56]:
import cv2

def extract_frames(video_path, output_dir, frame_rate=5):
    """
    Extract frames from a video at a specified frame rate.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame at the specified frame rate
        if frame_count % frame_rate == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    return saved_count



In [57]:
# Example: Extract frames for action_1
video_path = "soccer_extract_data/train/action_1.mp4"
output_dir = "soccer_extract_data/train/frames/action_1"
extract_frames(video_path, output_dir)

0

Feature Extraction


In [58]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(frame_paths):
    """
    Extract features from a list of frame paths.
    """
    features = []
    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        frame = cv2.resize(frame, (224, 224))  # Resize to ResNet input size
        frame = preprocess_input(frame)  # Preprocess for ResNet
        frame_features = base_model.predict(np.expand_dims(frame, axis=0))
        features.append(frame_features)
    return np.array(features)



In [59]:
# Example: Extract features for action_1
video_frame_dir = "soccer_extract_data/train/frames/action_1"
frame_paths = [os.path.join(video_frame_dir, f) for f in os.listdir(video_frame_dir)]
video_features = extract_features(frame_paths)

# Aggregate features (e.g., average pooling)
video_feature_vector = np.mean(video_features, axis=0)
print("Video feature vector shape:", video_feature_vector.shape)


In [67]:
!ls -lt

total 20
drwxr-xr-x 3 root root 4096 Feb 28 22:52 frames
drwxr-xr-x 6 root root 4096 Feb 28 22:26 soccer_extract_data
drwxr-xr-x 3 root root 4096 Feb 28 19:36 path
drwx------ 6 root root 4096 Feb 28 19:28 drive
drwxr-xr-x 1 root root 4096 Feb 27 14:22 sample_data


In [72]:
!ls -lt frames/

total 4
drwxr-xr-x 2 root root 4096 Feb 28 22:52 action_1


In [ ]:
def extract_frames(video_path, output_dir, frame_rate=5):
    """
    Extract frames from a video at a specified frame rate.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame at the specified frame rate
        if frame_count % frame_rate == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    return saved_count



In [73]:
# Example: Extract frames for all videos in the train directory
train_video_dir = "soccer_extract_data/train"  # Path to your train videos
train_frame_dir = "soccer_extract_data/train/frames"  # Where to save frames

# Iterate through all video files in the train directory
for video_file in os.listdir(train_video_dir):
    # Check if the file is a video file (e.g., .mp4)
    if video_file.endswith((".mp4", ".avi", ".mov")):  # Add other video extensions if needed
        video_path = os.path.join(train_video_dir, video_file)

        # Create a subdirectory for the video's frames
        video_name = os.path.splitext(video_file)[0]  # Get video name without extension
        output_dir = os.path.join(train_frame_dir, video_name)
# Extract frames
        extract_frames(video_path, output_dir)
        print(f"Extracted frames for {video_file} to {output_dir}")

In [74]:
!ls -lt soccer_extract_data/train/frames

total 11668
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1208
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1292
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_332
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1064
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_152
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2136
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2581
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_379
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_565
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1042
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1241
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1896
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1898
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2167
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_2837
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1399
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1581
drwxr-xr-x 2 root root 4096 Mar  1 01:42 action_1707
drwxr-xr-x 2 root root 4096 Mar  1 01:

In [75]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(frame_paths):
    """
    Extract features from a list of frame paths using ResNet50.
    """
    features = []
    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        frame = cv2.resize(frame, (224, 224))  # Resize to ResNet input size
        frame = preprocess_input(frame)  # Preprocess for ResNet
        frame_features = base_model.predict(np.expand_dims(frame, axis=0))
        features.append(frame_features)
    return np.array(features)



In [76]:
# Example usage:
train_frame_dir = "soccer_extract_data/train/frames"  # Directory where frames are stored

for video_name in os.listdir(train_frame_dir):
    video_frame_dir = os.path.join(train_frame_dir, video_name)
    frame_paths = [os.path.join(video_frame_dir, f) for f in os.listdir(video_frame_dir) if f.endswith(".jpg")]

    video_features = extract_features(frame_paths)

    # Aggregate features (e.g., average pooling)
    video_feature_vector = np.mean(video_features, axis=0)
    print(f"Features extracted for video {video_name}: Shape - {video_feature_vector.shape}")

    # Now you can use 'video_feature_vector' for further processing or training

Features extracted for video action_138: Shape - ()
Features extracted for video action_2559: Shape - ()
Features extracted for video action_1005: Shape - ()
Features extracted for video action_2654: Shape - ()
Features extracted for video action_1703: Shape - ()
Features extracted for video action_2740: Shape - ()
Features extracted for video action_2753: Shape - ()
Features extracted for video action_2095: Shape - ()
Features extracted for video action_2307: Shape - ()
Features extracted for video action_2690: Shape - ()
Features extracted for video action_1084: Shape - ()
Features extracted for video action_272: Shape - ()
Features extracted for video action_1776: Shape - ()
Features extracted for video action_572: Shape - ()
Features extracted for video action_2879: Shape - ()
Features extracted for video action_2476: Shape - ()
Features extracted for video action_2511: Shape - ()
Features extracted for video action_1227: Shape - ()
Features extracted for video action_1817: Shape -

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [77]:
# Load annotations
train_annotation_path = "soccer_extract_data/train/annotations.json"
with open(train_annotation_path, 'r') as f:
    train_annotations = json.load(f)

# Extract features and labels
train_frame_dir = "soccer_extract_data/train/frames"
features_dict = {} # Dictionary to store features and labels
labels = []

for video_name in os.listdir(train_frame_dir):
    video_frame_dir = os.path.join(train_frame_dir, video_name)
    frame_paths = [os.path.join(video_frame_dir, f) for f in os.listdir(video_frame_dir) if f.endswith(".jpg")]

    video_features = extract_features(frame_paths)
    video_feature_vector = np.mean(video_features, axis=0)  # Aggregate features

    features_dict[video_name] = video_feature_vector # Store features

    # Get label from annotations (assuming video_name matches the key in annotations)
    label = train_annotations.get(video_name, None) # Handle cases where key might not be found
    if label is not None:
        labels.append(label)
    else:
        print(f"Warning: Label not found for video {video_name}")

# Save the features and labels
np.savez_compressed("train_features.npz", features=features_dict, labels=labels)
print("Features and labels saved to train_features.npz")

Features and labels saved to train_features.npz


Loading the data:

You can load the data later like this:

In [ ]:
# ... (Your other functions: load_annotations, extract_frames, extract_features) ...


def save_features_and_labels(features, metadata, output_dir):
    """
    Saves the extracted features and labels to a file.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save features
    feature_path = os.path.join(output_dir, "features.npy")
    np.save(feature_path, features)

    # Save labels (metadata)
    label_path = os.path.join(output_dir, "metadata.json")
    with open(label_path, 'w') as f:
        json.dump(metadata, f, indent=4)


def process_videos(annotation_path, video_dir, output_dir, frame_rate=5):
    """
    Process all videos in the dataset:
    1. Extract frames.
    2. Extract features.
    3. Save features and labels.
    """
    annotations = load_annotations(annotation_path)
    all_features = []
    all_labels = []

    for action_id, metadata in annotations.items():
        video_path = os.path.join(video_dir, f"action_{action_id}.mp4")
        frame_dir = os.path.join(output_dir, "frames", f"action_{action_id}")
        processed_dir = os.path.join(output_dir, "processed", f"action_{action_id}")

        # Step 1: Extract frames
        extract_frames(video_path, frame_dir, frame_rate)

        # Step 2: Extract features
        frame_paths = [os.path.join(frame_dir, f) for f in os.listdir(frame_dir)]
        features = extract_features(frame_paths)
        feature_vector = np.mean(features, axis=0)

        # Step 3: Save features and labels
        save_features_and_labels(feature_vector, metadata, processed_dir) # Call the defined function

        # Collect features and labels for the full dataset
        all_features.append(feature_vector)
        all_labels.append(metadata)

    # Save the full dataset
    np.save(os.path.join(output_dir, "all_features.npy"), np.array(all_features))
    np.save(os.path.join(output_dir, "all_labels.npy"), np.array(all_labels))

In [81]:
def process_videos(annotation_path, video_dir, output_dir, frame_rate=5):
    """
    Process all videos in the dataset:
    1. Extract frames.
    2. Extract features.
    3. Save features and labels.
    """
    annotations = load_annotations(annotation_path)
    all_features = []
    all_labels = []

    for action_id, metadata in annotations.items():
        video_path = os.path.join(video_dir, f"action_{action_id}.mp4")
        frame_dir = os.path.join(output_dir, "frames", f"action_{action_id}")
        processed_dir = os.path.join(output_dir, "processed", f"action_{action_id}")

        # Step 1: Extract frames
        extract_frames(video_path, frame_dir, frame_rate)

        # Step 2: Extract features
        frame_paths = [os.path.join(frame_dir, f) for f in os.listdir(frame_dir)]
        features = extract_features(frame_paths)
        feature_vector = np.mean(features, axis=0)

        # Step 3: Save features and labels
        save_features_and_labels(feature_vector, metadata, processed_dir)

        # Collect features and labels for the full dataset
        all_features.append(feature_vector)
        all_labels.append(metadata)

    # Save the full dataset
    np.save(os.path.join(output_dir, "all_features.npy"), np.array(all_features))
    np.save(os.path.join(output_dir, "all_labels.npy"), np.array(all_labels))



In [80]:
# Example: Process all videos in the train directory
train_annotation_path = "soccer_extract_data/train/annotations.json"
train_video_dir = "soccer_extract_data/train/videos"
train_output_dir = "soccer_extract_data/train/processed"
process_videos(train_annotation_path, train_video_dir, train_output_dir)

NameError: name 'save_features_and_labels' is not defined

In [78]:
loaded_data = np.load("train_features.npz", allow_pickle=True)
features = loaded_data["features"].item()  # Access features dictionary
labels = loaded_data["labels"]  # Access labels array